<a href="https://colab.research.google.com/github/bhyeon1/Haribo_CNN/blob/main/Haribo_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_path = '/content/drive/MyDrive/Colab Notebooks/haribo_data'

In [7]:
data_path

'/content/drive/MyDrive/Colab Notebooks/haribo_data'

In [8]:
# 경로 설정
data_dir = '/content/drive/MyDrive/Colab Notebooks/haribo_data'

# 전처리 및 증강
img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=(0.8, 1.2),
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 545 images belonging to 5 classes.
Found 135 images belonging to 5 classes.


In [9]:
# 전이학습 모델 로드 및 일부 층만 fine-tune
base_model = MobileNetV2(input_shape=img_size + (3,),
                         include_top=False,
                         weights='imagenet')

base_model.trainable = True  # 전체 레이어 훈련 가능

# 앞부분은 동결, 뒷부분만 훈련
for layer in base_model.layers[:-30]:
    layer.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
# 전체 모델 구성
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.4),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

In [11]:
# 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # 낮은 lr 사용
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 콜백
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

In [ ]:
# 학습
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[early_stop, lr_scheduler],
    verbose=2
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
18/18 - 200s - 11s/step - accuracy: 0.6569 - loss: 0.9058 - val_accuracy: 0.7185 - val_loss: 0.8457 - learning_rate: 1.0000e-04
Epoch 2/30
18/18 - 16s - 863ms/step - accuracy: 0.9835 - loss: 0.1199 - val_accuracy: 0.8519 - val_loss: 0.4680 - learning_rate: 1.0000e-04
Epoch 3/30
18/18 - 16s - 874ms/step - accuracy: 0.9835 - loss: 0.0822 - val_accuracy: 0.8963 - val_loss: 0.3474 - learning_rate: 1.0000e-04
Epoch 4/30
18/18 - 16s - 881ms/step - accuracy: 0.9927 - loss: 0.0473 - val_accuracy: 0.8963 - val_loss: 0.2903 - learning_rate: 1.0000e-04
Epoch 5/30
18/18 - 16s - 893ms/step - accuracy: 0.9927 - loss: 0.0379 - val_accuracy: 0.9407 - val_loss: 0.2118 - learning_rate: 1.0000e-04
Epoch 6/30
18/18 - 17s - 971ms/step - accuracy: 0.9945 - loss: 0.0337 - val_accuracy: 0.9481 - val_loss: 0.1953 - learning_rate: 1.0000e-04
Epoch 7/30
18/18 - 16s - 864ms/step - accuracy: 0.9982 - loss: 0.0228 - val_accuracy: 0.9704 - val_loss: 0.1356 - learning_rate: 1.0000e-04
Epoch 8/30
18/18 - 16

In [ ]:
# 평가
val_loss, val_acc = model.evaluate(val_gen, verbose=0)
print(f"✅ Fine-tuned Validation Accuracy: {val_acc:.4f}")

In [ ]:
# 시각화
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
print(train_generator.class_indices)

In [ ]:
model.save("haribo_model.h5")

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

model.save("/content/drive/MyDrive/haribo_model.h5")